In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

In [9]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import xarray as xa
import rioxarray as rio
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from matplotlib.figure import Figure
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from pathlib import Path
import pathlib
from tqdm import tqdm
from scipy.ndimage import binary_dilation

import netCDF4

from coralshift.utils import file_ops, directories
from coralshift.dataloading import bathymetry
from coralshift.processing import data

In [3]:
all_climate = file_ops.load_merge_nc_files("/Volumes/MRes Drive/global_ocean_reanalysis/monthly_means/baseline_area")

In [5]:
time = xa.cftime_range(start='2022-01-01', end='2022-01-05', freq='D')

# create lat and lon coordinates
lat = np.linspace(-90, 90, 5)
lon = np.linspace(-180, 180, 5)

# create random data for each coordinate
# create random data for each variable and coordinate
data1 = np.random.rand(len(time), len(lat), len(lon))
data2 = np.random.rand(len(time), len(lat), len(lon))
data3 = np.random.rand(len(time), len(lat), len(lon))

# create xarray dataset with time, lat, and lon as coordinates
ds = xa.Dataset({'data1': (['time', 'lat', 'lon'], data1),
                 'data2': (['time', 'lat', 'lon'], data2),
                 'data3': (['time', 'lat', 'lon'], data3)},
                coords={'time': time,
                        'lat': lat,
                        'lon': lon})


# stacked.transpose('time','points').as_numpy()

# np.shape(stacked['data1'].transpose('time','points').to_numpy())

def stacking(ds):
    stacked = ds.stack(points=('lat', 'lon'))
    
    vars_list = list(ds.variables)
    # stack= stacked[vars_list[0]].to_numpy()
    vals = []
    for d in vars_list[:3]:
        vals.append(stacked[d].to_numpy())
        # vals = stacked[d].transpose('time','points').to_numpy()
        # stack = np.stack([stack, vals], axis=1)

    return vals

out = stacking(ds)

In [7]:
ds

<xarray.Dataset>
Dimensions:  (time: 5, lat: 5, lon: 5)
Coordinates:
  * time     (time) object 2022-01-01 00:00:00 ... 2022-01-05 00:00:00
  * lat      (lat) float64 -90.0 -45.0 0.0 45.0 90.0
  * lon      (lon) float64 -180.0 -90.0 0.0 90.0 180.0
Data variables:
    data1    (time, lat, lon) float64 0.6346 0.9547 0.5269 ... 0.03847 0.2178
    data2    (time, lat, lon) float64 0.6083 0.4093 0.3924 ... 0.8859 0.6386
    data3    (time, lat, lon) float64 0.648 0.7405 0.1738 ... 0.2638 0.3161

In [31]:
ds_stacked = ds.stack(location=('lat', 'lon'))
ds_stacked


<xarray.Dataset>
Dimensions:   (time: 5, location: 25)
Coordinates:
  * time      (time) object 2022-01-01 00:00:00 ... 2022-01-05 00:00:00
  * location  (location) object MultiIndex
  * lat       (location) float64 -90.0 -90.0 -90.0 -90.0 ... 90.0 90.0 90.0 90.0
  * lon       (location) float64 -180.0 -90.0 0.0 90.0 ... -90.0 0.0 90.0 180.0
Data variables:
    data1     (time, location) float64 0.6346 0.9547 0.5269 ... 0.03847 0.2178
    data2     (time, location) float64 0.6083 0.4093 0.3924 ... 0.8859 0.6386
    data3     (time, location) float64 0.648 0.7405 0.1738 ... 0.2638 0.3161

In [34]:
list(ds.variables)

['data1', 'data2', 'data3', 'time', 'lat', 'lon']

In [78]:



temp = []
for dim in list(ds.variables)[:3]:
    vals = ds_stacked[dim].values
    temp.append(vals)

# move location to first column
swapped=np.moveaxis(temp,2,0)


(25, 3, 5)

In [69]:
np.shape(temp)

(3, 5, 25)

(25, 3, 5)

In [70]:
np.shape(swapped[:,:,4])

(25, 3)

In [20]:
data1_array = ds_stacked['data1'].values
np.shape(data1_array)

(5, 25)

In [50]:
concatenated = np.concatenate(temp, axis=1)

In [51]:
np.shape(concatenated)

(5, 75)

In [ ]:
transposed = np.transpose(out, (2, 1, 0))
reshaped = np.reshape(transposed, (25, 5, 3))


In [ ]:
np.shape(reshaped[2])

In [ ]:
out[0]

# Climate data imports

### Climatology Monthly Means
[Source](https://data.marine.copernicus.eu/product/GLOBAL_MULTIYEAR_PHY_001_030/services): ```cmems_mod_glo_phy_my_0.083-climatology_P1M-m```

Contains the average variable values in each month of the year between 1993 and 2020.

In [79]:
data_dict = file_ops.dict_of_ncs_from_dir("/Volumes/MRes Drive/global_ocean_reanalysis/climatology_monthly_means", engine="h5netcdf")
data_dict["mercatorglorys12v1_gl12_mean_1993_2016_05"]

100%|██████████| 12/12 [00:12<00:00,  1.05s/it]


<xarray.Dataset>
Dimensions:      (time: 1, longitude: 4320, latitude: 2041, depth: 50)
Coordinates:
  * time         (time) datetime64[ns] 1993-05-16T12:00:00
  * longitude    (longitude) float32 -180.0 -179.9 -179.8 ... 179.8 179.8 179.9
  * latitude     (latitude) float32 -80.0 -79.92 -79.83 ... 89.83 89.92 90.0
  * depth        (depth) float32 0.494 1.541 2.646 ... 5.275e+03 5.728e+03
    spatial_ref  int64 0
Data variables:
    mlotst       (time, latitude, longitude) float32 ...
    zos          (time, latitude, longitude) float32 ...
    bottomT      (time, latitude, longitude) float32 ...
    sithick      (time, latitude, longitude) float32 ...
    siconc       (time, latitude, longitude) float32 ...
    usi          (time, latitude, longitude) float32 ...
    vsi          (time, latitude, longitude) float32 ...
    thetao       (time, depth, latitude, longitude) float32 ...
    so           (time, depth, latitude, longitude) float32 ...
    uo           (time, depth, latitude, longitude) float32 ...
    vo           (time, depth, latitude, longitude) float32 ...
Attributes: (12/16)
    CDI:                           Climate Data Interface version 1.9.3 (http...
    source:                        MERCATOR GLORYS12V1
    institution:                   Mercator Ocean
    Conventions:                   CF-1.6
    references:                    http://marine.copernicus.eu
    credit:                        E.U. Copernicus Marine Service Information...
    ...                            ...
    product:                       GLOBAL_REANALYSIS_001_030
    product_user_manual:           http://marine.copernicus.eu/documents/PUM/...
    quality_information_document:  http://marine.copernicus.eu/documents/QUID...
    CDO:                           Climate Data Operators version 1.9.3 (http...
    title:                         Monthly climatology fields for product GLO...
    dataset:                       global-reanalysis-phy-001-030-monthly-clim...

In [ ]:
(data_dict["mercatorglorys12v1_gl12_mean_1993_2016_02"]).dims

In [ ]:
# flat_vars = ["zos","mlotst","bottomT","sithick","siconc","usi","vsi"]

# one = data_dict["mercatorglorys12v1_gl12_mean_1993_2016_02"][flat_vars]
# two = data_dict["mercatorglorys12v1_gl12_mean_1993_2016_03"][flat_vars]
# out = xa.concat([one,two], dim="time")
# out

In [ ]:
f,ax1=plt.subplots()
out['zos'][0].plot(ax=ax1)
ax1.axis('equal');

In [ ]:
flat_vars = ["zos","mlotst","bottomT","sithick","siconc","usi","vsi"]

month_datasets = [data_dict[month][flat_vars] for month in sorted(data_dict.keys())]

In [ ]:
for ds in month_datasets:
    print(ds.dims)

In [ ]:
month_datasets[:1][0]

In [ ]:
one = data_dict["mercatorglorys12v1_gl12_mean_1993_2016_02"][flat_vars]
two = data_dict["mercatorglorys12v1_gl12_mean_1993_2016_01"][flat_vars]
xa.concat([one,two], dim="time")

In [ ]:
# 1D variables
flat_vars = ["zos","mlotst","bottomT","sithick","siconc","usi","vsi"]

month_datasets = [data_dict[month][flat_vars] for month in sorted(data_dict.keys())]
# month_datasets[2]
flat_vars_ds = xa.concat(month_datasets, dim='time')

In [ ]:
flat_vars_ds

In [ ]:
# f,ax1=plt.subplots()
# surface_monthly_means['zos'][0].plot(ax=ax1)
# ax1.axis('equal');


In [ ]:
import numpy as np
import xarray as xr

# create x and y coordinates
x = np.arange(0, 4320, 1)
y = np.arange(0, 20410, 1)

# create time coordinate
da1_time = np.arange('2022-01', '2022-03', dtype='datetime64[M]')
da1_time = np.arange('2022-03', '2022-04', dtype='datetime64[M]')


# create data for the first DataArray
data1 = np.random.rand(len(time), len(y), len(x))

# create the first DataArray
da1 = xr.DataArray(data1, coords=[time, y, x], dims=['time', 'y', 'x'])

# create data for the second DataArray
data2 = np.random.rand(len(time), len(y), len(x))

# create the second DataArray
da2 = xr.DataArray(data2, coords=[time, y, x], dims=['time', 'y', 'x'])


In [ ]:
xa.concat([da1, da2], dim="time")

In [ ]:
month_datasets = [data_dict[month] for month in sorted(data_dict.keys())]
# month_datasets[2]
concat_data = xa.concat(month_datasets[2:3], dim='time')
# sorted_data = concat_data.sortby('time')


In [ ]:
fig, ax = plt.subplots(12, 1)

for i, array in tqdm(enumerate(out.values())):
    # upsampled_array = upsample_xa_array(array, resolution=1)
    array["bottomT"].plot(ax=ax[i])

# upsampled_array["bottomT"].plot(ax=ax[i])

In [ ]:
fig, ax = plt.subplots(len(out), 1)

for i, array in tqdm(enumerate(out.values())):
    array["mlotst"].plot(ax=ax[i])

In [ ]:
upsample_xa_array(out["mercatorglorys12v1_gl12_mean_1993_2016_05"]['mlotst'])

In [ ]:
(upsample_xa_array(out["mercatorglorys12v1_gl12_mean_1993_2016_05"]['mlotst'])).plot()

In [ ]:
out["mercatorglorys12v1_gl12_mean_1993_2016_08"].variables.keys()

In [ ]:
out["mercatorglorys12v1_gl12_mean_1993_2016_07"].variables["usi"][:][0]

## Ocean Acidification

In [ ]:
acids = read_ncs_from_dir("/Volumes/MRes Drive/ocean_acidification")

In [ ]:
acids["Aragonite_std_historical"]